In [22]:
import pandas as pd
from pyliftover import LiftOver
df = pd.read_csv("../data/variant_summary.txt", delimiter="\t")
df = df[["GeneSymbol", "Start", "Stop", "Chromosome", "ClinSigSimple", "Type"]]
df = df[df['GeneSymbol'] == "BRCA1"]

df = df[df['Start'] >= 0]

lo = LiftOver('hg19', 'hg38')
gene_start_position_hg38 = 43044292

hg19_rows = df[df['Start'] < gene_start_position_hg38]
hg19_rows['Start'] = [lo.convert_coordinate('chr17', start)[0][1] for start in hg19_rows['Start']]
hg19_rows['Stop'] = [lo.convert_coordinate('chr17', stop)[0][1] for stop in hg19_rows['Stop']]
df = df[df['Start'] >= gene_start_position_hg38]
df = pd.concat([df, hg19_rows])
    

/var/folders/y2/hmyqc1_n72j7cfjnz6w3k4rm0000gn/T/ipykernel_68860/1187327113.py:3: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/variant_summary.txt", delimiter="\t")
/var/folders/y2/hmyqc1_n72j7cfjnz6w3k4rm0000gn/T/ipykernel_68860/1187327113.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hg19_rows['Start'] = [lo.convert_coordinate('chr17', start)[0][1] for start in hg19_rows['Start']]
/var/folders/y2/hmyqc1_n72j7cfjnz6w3k4rm0000gn/T/ipykernel_68860/1187327113.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

In [23]:
import pyBigWig

bw = pyBigWig.open("../data/hg38.phyloP100way.bw")

chrom = "chr17"   # UCSC style: add "chr" prefix

# pyBigWig uses 0-based, half-open intervals
scores = [bw.values(chrom, row['Start']-1, row['Stop'])[0] for _, row in df.iterrows()]


In [24]:
scores

[4.877999782562256,
 5.0980000495910645,
 0.4490000009536743,
 1.274999976158142,
 -0.5220000147819519,
 -0.7360000014305115,
 -1.3389999866485596,
 -0.02500000037252903,
 1.1619999408721924,
 0.5049999952316284,
 0.4790000021457672,
 0.45500001311302185,
 1.5269999504089355,
 1.5269999504089355,
 -0.17800000309944153,
 -0.21699999272823334,
 1.3489999771118164,
 -0.02199999988079071,
 -0.16599999368190765,
 0.5789999961853027,
 2.4649999141693115,
 4.159999847412109,
 5.026000022888184,
 5.026000022888184,
 0.07599999755620956,
 0.2409999966621399,
 0.47999998927116394,
 0.45100000500679016,
 0.8029999732971191,
 2.309999942779541,
 3.4749999046325684,
 3.4749999046325684,
 0.6850000023841858,
 -0.03999999910593033,
 1.0759999752044678,
 1.1799999475479126,
 -1.0570000410079956,
 -0.15600000321865082,
 2.055000066757202,
 -0.020999999716877937,
 0.4749999940395355,
 -0.44200000166893005,
 4.26800012588501,
 1.934000015258789,
 -0.23800000548362732,
 3.384999990463257,
 3.3849999904632

In [25]:
df['PhyloScore'] = scores

In [26]:
df.to_csv('../data/BRCA1_phylo_scores.csv', index=False)